# Métodos para Resolução de Sistemas Lineares #

Nesta aula, iremos implementar métodos para encontrar vetores de solução $x$ para sistemas lineares no formato $Ax=b$.

# Preliminares #

### Resolvendo sistemas triangulares ###

Normalmente, para resolver sistemas lineares, precisamos chegar de alguma forma a sistemas triangulares (inferiores ou superiores). Por que? Porque eles podem ser resolvidos pelo método das substituições retroativas.



$$ x_{i} = \frac{b_{i}-\sum\limits_{j=1}^{i-1}{a_{ij}x_{j}}}{a_{ii}} $$

e

$$ x_{i} = \frac{b_{i}-\sum\limits_{j=i+1}^{n}{a_{ij}x_{j}}}{a_{ii}} $$

Faça duas funções, que recebendo como parâmetro uma matriz $A$ triangular (superior ou inferior) e um vetor $b$, retorne o vetor solução $X$:

In [1]:
import numpy as np

In [64]:
# Dão erro se tiver 0 na DP pq aí é indeterminado
def resolveTS(A,b): #A Triangular Superior
    x = np.zeros(len(A))
    for i in range(len(A)-1,-1,-1):
        x[i] = (b[i] - (A[i][i+1:]*x[i+1:]).sum())/A[i][i]
    return x

def resolveTI(A,b): #A Triangular Inferior
    x = np.zeros(len(b))
    for i in range(len(b)):
        x[i] = (b[i] - (A[i][:i]*x[:i]).sum())/A[i][i] # x[<intervalo_fechado>:<intervalo_aberto>]
    return x

In [65]:
A = np.array([[1,1,-2,0],[0,4,0,1],[0,0,3,7/4],[0,0,0,-5/3]])
b = [5,11,9/4,-5]
resolveTS(A,b)

array([ 1.,  2., -1.,  3.])

In [66]:
A1 = np.flip(np.flip(A,0),1)
b1 = np.flip(b,0)
resolveTI(A1,b1)

array([ 3., -1.,  2.,  1.])

### Operações $l$-elementares ###

Fazer operações $l$-elementares com matrizes numpy é bem simples: com `M[i]` você acessa a i-ésima linha:

In [68]:
import numpy as np

M = np.random.randint(-10,10,(4,4))
print(M)

print("Subtraindo uma linha por outra multiplicada por uma constante:")
M[2] = M[2] - M[1]*2
print(M)

print("Trocar duas linhas:")

M[[1,3]] = M[[3,1]]
print(M)


[[ -2  -5  -2   5]
 [  1   2   9   7]
 [  9   4  -9   5]
 [  1 -10  -4 -10]]
Subtraindo uma linha por outra multiplicada por uma constante:
[[ -2  -5  -2   5]
 [  1   2   9   7]
 [  7   0 -27  -9]
 [  1 -10  -4 -10]]
Trocar duas linhas:
[[ -2  -5  -2   5]
 [  1 -10  -4 -10]
 [  7   0 -27  -9]
 [  1   2   9   7]]


In [11]:
print("Achar o maior elemento da matriz:")
print(M[1].max())

print("Achar o maior elemento da matriz em módulo:")
print(abs(M[:][2]).max())

# E PARA ACHAR O MAIOR ELEMENTO DE UMA LINHA?
print(M[0][:].max())

Achar o maior elemento da matriz:
-1
Achar o maior elemento da matriz em módulo:
20
7


In [12]:
print("Matriz transposta:\n",M[1:3].T)

print("Determinante de M: ",np.linalg.det(M))

print("Autovalores de uma matriz:")

(a,_) = np.linalg.eig(M) 
print(a)

Matriz transposta:
 [[ -1  20]
 [ -5 -19]
 [ -7  -4]
 [ -1   2]]
Determinante de M:  -9399.999999999995
Autovalores de uma matriz:
[ 14.77990546 -18.54949633  -9.6932492   -3.53715994]


# Métodos Exatos #

## Eliminação gaussiana ##

Para fazer a eliminação gaussiana deve-se usar os elementos da diagonal principal de A como pivos para zerar os elementos da mesma coluna.

Implemente a eliminação gaussiana simples (sem pivotação parcial):

In [169]:
def eliminacaoGaussianaSimples(A0,b):
    M = np.zeros(A0.shape)
    A = np.concatenate((A0,b.reshape(len(b),1)),axis=1)
    for i in range(len(A)-1):
        M[i+1:,i] = A[i+1:,i]/A[i][i]
        A[i+1:] -= M[i+1:,i].reshape((len(M[i+1:]),1))*A[i]
    return resolveTS(A[:,:len(b)],A[:,len(b)])

In [60]:
A = np.array([[1,1,-2,0],[-1,3,2,1],[1,2,1,2],[2,0,-2,-1]],dtype='float')
b = np.array([5,6,10,1],dtype='float')
eliminacaoGaussianaSimples(A,b)

array([ 1.,  2., -1.,  3.])

## Decomposição LU ##

A decomposição LU é mais utilizada quando a mesma matriz de coeficientes $A$ é usada para várias soluções diferentes. Por isto, ela pode ser dividida em dois passos:

- Decompor $A$ em $L$ e $U$ 
- Dados $L$, $U$ e $b$, achar a solução X

Faça as duas funções, com a decomposição ainda sem pivotação parcial:

In [249]:
def decompoeLU(A):
    L = np.identity(len(A))
    U = A.copy()
    for i in range(len(A)-1):
        L[i+1:,i] = U[i+1:,i]/U[i][i]
        U[i+1:] -= L[i+1:,i].reshape((len(L[i+1:]),1))*U[i]
    return L,U 

def resolveLU(L,U,b):
    y = resolveTI(L,b)
    return resolveTS(U,y)

def LU(A,b):
    L,U = decompoeLU(A)
    return resolveLU(L,U,b)

In [250]:
A = np.array([[1,6,2,4],[3,19,4,15],[1,4,8,-12],[5,33,9,3]],dtype='float')
b = np.array([8,25,18,72],dtype='float')
LU(A,b)

array([-138.,   20.,   11.,    1.])

### Pivotação Parcial ###

Sistemas onde o determinante de uma das submatrizes principais ($A_{1x1},A_{2x2},A_{3x3}...$) é igual a $0$ não podem ser resolvidos com a decomposição LU simples. Nestes casos, deve-se utilizar a pivotação parcial, onde o pivô é escolhido da linha com o maior elemento em módulo.

Contudo, é importante guardar as trocas de linha que foram efetuadas na matriz de permutações $P$. Esta matriz é uma matriz identidade com as linhas trocadas junto com a pivotação. Por exemplo, se na primeira coluna o maior elemento está na linha três, este será o primeiro pivô (a linha 1 será trocada com a 3). Neste caso, na matriz $P$ também se troca estas linhas. No fim do processo:

$PAx = Pb$

$LUx=Pb$

Então basta resolver trocando as linhas de b de através da multiplicação por P. Lembrem que multiplicar matrizes em numpy é:

`p.dot(a)` ou `dot(p,a)`

Implemente uma função que checa se a pivotação parcial é necessária e a decomposição LU com pivotação parcial:

In [313]:
def verificaPivot(A):
    for i in range(len(A)):
        if np.linalg.det(A[:i,:i]) == 0:
            return False
    return True

def LUparcial(A):
    P = np.identity(len(A))
    U = A.copy()
    if verificaPivot(A):
        L = np.identity(len(A))
        for i in range(len(A)-1):
            lp = np.argmax(np.abs(U[i:,i])) + i # Pega o índice da linha pivotal
            U[[i,lp]] = U[[lp,i]] # Troca as linhas na matriz A
            P[[i,lp]] = P[[lp,i]] # Troca as linhas na matriz P
            L[i+1:,i] = U[i+1:,i]/U[i][i] # Coloca a coluna de multiplicadores em L
            U[i+1:] -= L[i+1:,i].reshape((len(L[i+1:]),1))*U[i]
            U[i+1:] = U[i+1:].round(decimals=2)
            L[i+1:,i] = L[i+1:,i].round(decimals=2)
            #print(U)
        return L,U,P
    else:
        print('Não é possível fazer a pivotação parcial.')
        return []

def resolveLUpar(L,U,P,b):
    y = resolveTI(L,np.dot(P,b))
    return resolveTS(U,y)

def LUpar(A,b):
    L,U,P = LUparcial(A)
    return resolveLUpar(L,U,P,b)

In [314]:
A = np.array([[1,6,2,4],[3,19,4,15],[1,4,8,-12],[5,33,9,3]],dtype='float')
b = np.array([8,25,18,72],dtype='float')
LUpar(A,b)

array([-10912.76117499,   1379.36203695,    951.1806284 ,    185.411     ])

#### Exercício ####

Rode e verifique o tempo com `%timeit` da eliminação gaussiana, decomposição LU e LU com pivotação para o sistema abaixo. No caso da LU, calcule o tempo da decomposição e da solução dos sistemas:

In [315]:
#Sistema 1:
print('Sistema 1:')
M1 = np.array([[1,-3,5,6], [-8,4,-1,0],[3,2,-2,7],[1,2,5,-4]],dtype='float')
b1 = np.array([17,29,-11,7],dtype='float')
print('Eliminação Gaussianda Simples: \t       ',end='')
%timeit -n100 eliminacaoGaussianaSimples(M1,b1)
print('Decomposição LU:\t\t       ',end='')
%timeit -n100 LU(M1,b1)
print('Decomposição LU com pivotação parcial: ',end='')
%timeit -n100 LUpar(M1,b1)

#Sistema 2:
print('\nSistema 2:')
M2 = np.array([[-2,3,1,5],[5,1,-1,0],[1,6,3,-1],[4,5,2,8]],dtype='float')
b2 = np.array([2,-1,0,6],dtype='float')
print('Eliminação Gaussianda Simples: \t       ',end='')
%timeit -n100 eliminacaoGaussianaSimples(M2,b2)
print('Decomposição LU:\t\t       ',end='')
%timeit -n100 LU(M2,b2)
print('Decomposição LU com pivotação parcial: ',end='')
%timeit -n100 LUpar(M2,b2)


Sistema 1:
Eliminação Gaussianda Simples: 	       142 µs ± 34.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Decomposição LU:		       138 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Decomposição LU com pivotação parcial: 381 µs ± 64.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Sistema 2:
Eliminação Gaussianda Simples: 	       90.3 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Decomposição LU:		       151 µs ± 35.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Decomposição LU com pivotação parcial: 549 µs ± 230 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Cholesky ##

O método de Cholesky só pode ser utilizado quando a matriz for:

- Simétrica (igual a sua transposta)
- Definida positiva: 
    - Todos os elementos da diagonal principal são positivos
    - Todos os autovalores de $A$ são positivos
    - Todas as submatrizes superiores possuem determinante __positivo__.

Se for possível, o método de Cholesky é uma decomposição LU onde $U=L^{T}$, ou seja $LL^{T}x=b$.

Lembrando que na decomposição de Cholesky os elementos da diagonal principal de L são:

$$l_{jj} = \sqrt{a_{jj}-\sum\limits_{k=1}^{j-1}{l_{jk}^2}, j = 1,2,...,n}$$

E os fora da diagonal principal são:

$$ l_{ij} = \frac{a_{ij}-\sum\limits_{k=1}^{j-1}{l_{ik}l_{jk}}}{l_{jj}} $$

Desta forma, faça uma função para determinar se uma matriz pode ser resolvida via Cholesky e uma para encontrar L (para resolver pode-se usar `resolveLU(L,L.T,b)`):

In [348]:
def verificaCholesky(A):
    return (A == A.T).all() and (A.diagonal() > 0).all() and (np.linalg.eig(A)[0] > 0).all() and verificaPivot(A)

def geraCholesky(A):
    L = np.zeros(A.shape)
    for j in range(len(A)):
        L[j][j] = np.sqrt(A[j][j] - (np.power(L[j][:j],2).sum()))
        L[j+1:,j] = (A[j+1:,j] - (L[j+1:,:j]*L[j,:j]).sum())/L[j][j]
    return L

def cholesky(A,b):
    L = geraCholesky(A)
    return resolveLU(L,L.T,b)

In [374]:
A = np.array([[4,-2,2],[-2,10,-7],[2,-7,30]],dtype='float')
verificaCholesky(A)

True

In [375]:
geraCholesky(A)

array([[ 2.,  0.,  0.],
       [-1.,  3.,  0.],
       [ 1., -2.,  5.]])

No método de Cholesky, por ser uma matriz simétrica, podemos, ao invés de calcular o determinante normalmente, usar a seguinte definição:

$$ det(A) = det(L)det(L') $$

$$ det(A) = \bigg(\prod_{i=1}^{n}{l_{ii}}\bigg)^2 $$

In [303]:
def detCholesky(M): 
    return np.power(np.prod(geraCholesky(M).diagonal()),2)

In [376]:
detCholesky(A)

900.0

#### Exercicios ####

1 - Verifique se os seguintes sistemas podem ser resolvidas via Choleski

2 - Compare o tempo para decomposição LU das que são possíveis com a de Choleski usando `%timeit`

3 - Compare o tempo do calculo do determinante de numpy com o determinante específico para matrizes para choleski

In [371]:
print('Sistema 3:')
M3 = np.array([[9,-6,3],[-6,29,-7],[3,-7,18]],dtype='float')
b3 = np.array([-3,-8,33],dtype='float')
print('É possível de se resolver com Sholesky?',verificaCholesky(M3))
print('Cholesky:\t ',end='')
%timeit -n100 cholesky(M3,b3)
print('Decomposição LU: ',end='')
%timeit -n100 LU(M3,b3)
print('Determinante Numpy:    ',end='')
%timeit -n100 np.linalg.det(M3)
print('Determinante Cholesky: ',end='')
%timeit -n100 detCholesky(M3)

print('\nSistema 4:')
M4 = np.array([[4,-2,4,10],[-2,2,-1,-7],[4,-1,14,11],[10,-7,11,31]],dtype='float')
b4 = np.array([2,2,-1,-2],dtype='float')
print('É possível de se resolver com Sholesky?',verificaCholesky(M4))
print('Cholesky:\t ',end='')
%timeit -n100 cholesky(M4,b4)
print('Decomposição LU: ',end='')
%timeit -n100 LU(M4,b4)
print('Determinante Numpy:    ',end='')
%timeit -n100 np.linalg.det(M4)
print('Determinante Cholesky: ',end='')
%timeit -n100 detCholesky(M4)

print('\nSistema 5:')
M5 = np.array([[16,-4,4,12],[-4,2,-1,-7],[4,-1,26,13],[12,-7,13,25]],dtype='float')
b5 = np.array([2,2,-1,-2],dtype='float')
print('É possível de se resolver com Sholesky?',verificaCholesky(M5))

Sistema 3:
É possível de se resolver com Sholesky? True
Cholesky:	 183 µs ± 44.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Decomposição LU: 109 µs ± 5.62 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Determinante Numpy:    9.69 µs ± 899 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
Determinante Cholesky: 119 µs ± 3.94 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Sistema 4:
É possível de se resolver com Sholesky? True
Cholesky:	 225 µs ± 29.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Decomposição LU: 149 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Determinante Numpy:    9.68 µs ± 914 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
Determinante Cholesky: 156 µs ± 6.87 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Sistema 5:
É possível de se resolver com Sholesky? False


/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in sqrt
  import sys
/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in sqrt
  import sys
